In [169]:
from hydradx.model.amm.omnipool_amm import OmnipoolState
from hydradx.model.amm.agents import Agent




In [170]:
dot_usdt_order_book = {
    'bids': [{'price': 3.80,'amount': 200},
             {'price': 3.79,'amount': 100},
             {'price': 3.70,'amount': 100}],
    'asks': [{'price': 3.90,'amount': 100},
             {'price': 4.00,'amount': 200}]
}
dot_usdt_order_book = {
    'bids': [{'price': 3.60, 'amount': 200},
             {'price': 3.59, 'amount': 100},
             {'price': 3.50, 'amount': 100},
             {'price': 3.40, 'amount': 2000}],
    'asks': [{'price': 3.70, 'amount': 100},
             {'price': 3.74, 'amount': 5000},
             {'price': 3.80, 'amount': 200},
             {'price': 3.90, 'amount': 2000}]
}

tokens = {
    'USDT': {
        'liquidity': 1000000,
        'LRNA': 100000
    },
    'DOT': {
        'liquidity': 800000,
        'LRNA': 300000
    },
    'HDX': {
        'liquidity': 200000000,
        'LRNA': 400000
    }
}

op_state = OmnipoolState(
            tokens=tokens,
            lrna_fee=0.00,
            asset_fee=0.00,
    preferred_stablecoin='USDT',
        )

init_state = op_state.copy()

OmnipoolState.price(op_state, 'DOT', 'USDT')

3.75

In [171]:

bids = dot_usdt_order_book['bids']
asks = dot_usdt_order_book['asks']
bids = sorted(bids, key=lambda x: x['price'], reverse=True)
asks = sorted(asks, key=lambda x: x['price'], reverse=False)

agent = Agent(holdings={'USDT': 1000000, 'DOT': 1000000, 'HDX': 1000000}, unique_id='bot')

op_spot = OmnipoolState.price(op_state, 'DOT', 'USDT')
i = 0
profit = 0
bid_executed = False
ask_executed = False
last_profit = 0
if op_spot < bids[i]['price']:
    bid_executed = True
    test_state = op_state
    while i < len(bids) and op_spot < bids[i]['price']:  # spot is lower than the highest bid, so we should buy from Omnipool and sell to CEX
        print('buy')
        op_state = test_state
        profit += last_profit
        init_usdt = agent.holdings['USDT']
        init_dot = agent.holdings['DOT']
        test_state = op_state.copy()
        test_state.swap(agent, tkn_buy='DOT', tkn_sell='USDT', buy_quantity=bids[i]['amount'])
        
        op_spot = OmnipoolState.price(test_state, 'DOT', 'USDT')
        usdt_in = init_usdt - agent.holdings['USDT']
        last_profit = bids[i]['price'] * bids[i]['amount'] - usdt_in
        i += 1
elif op_spot > asks[i]['price']:
    ask_executed = True
    test_state = op_state
    while i < len(bids) and op_spot > asks[i]['price']:  # spot is higher than the lowest ask, so we should buy from CEX and sell to Omnipool
        print('sell')
        op_state = test_state
        profit += last_profit
        init_usdt = agent.holdings['USDT']
        init_dot = agent.holdings['DOT']
        test_state = op_state.copy()
        test_state.swap(agent, tkn_buy='USDT', tkn_sell='DOT', sell_quantity=asks[i]['amount'])
        
        op_spot = OmnipoolState.price(test_state, 'DOT', 'USDT')
        usdt_out = agent.holdings['USDT'] - init_usdt 
        last_profit = usdt_out - asks[i]['price'] * asks[i]['amount'] 
        i += 1

swaps = []
if bid_executed:
    if op_spot < bids[i-1]['price']:  # last trade can be fully executed without moving spot price too much
        swaps = bids[0:i]
        profit += last_profit
    else:  # Use binary search to partially fill last order
        swaps = bids[0:i-1]
        bid = bids[i-1]
        amt_high = bid['amount']
        amt_low = 0
        iters = 100
        init_usdt = agent.holdings['USDT']
        init_dot = agent.holdings['DOT']
        for i in range(iters):
            test_state = op_state.copy()
            amt = amt_low + (amt_high - amt_low) / 2
            test_state.swap(agent, tkn_buy='DOT', tkn_sell='USDT', buy_quantity=amt)
            op_spot = OmnipoolState.price(test_state, 'DOT', 'USDT')
            usdt_in = init_usdt - agent.holdings['USDT']
            # profit = bid['price'] * amt - usdt_in
            if op_spot > bid['price']:
                amt_high = amt
            elif op_spot < bid['price']:
                amt_low = amt
            else:
                break
                
        op_state.swap(agent, tkn_buy='DOT', tkn_sell='USDT', buy_quantity=amt_low)
        op_spot = OmnipoolState.price(op_state, 'DOT', 'USDT')
        swaps.append({'price': bid['price'], 'amount': amt_low})
        usdt_in = init_usdt - agent.holdings['USDT']
        profit += bid['price'] * amt_low - usdt_in

elif ask_executed:
    if op_spot > asks[i-1]['price']:  # last trade can be fully executed without moving spot price too much
        swaps = asks[0:i]
        profit += last_profit
    else:  # Use binary search to partially fill last order
        swaps = asks[0:i-1]
        ask = asks[i-1]
        amt_high = ask['amount']
        amt_low = 0
        iters = 100
        init_usdt = agent.holdings['USDT']
        init_dot = agent.holdings['DOT']
        for i in range(iters):
            test_state = op_state.copy()
            amt = amt_low + (amt_high - amt_low) / 2
            test_state.swap(agent, tkn_buy='USDT', tkn_sell='DOT', sell_quantity=amt)
            op_spot = OmnipoolState.price(test_state, 'DOT', 'USDT')
            usdt_out = agent.holdings['USDT'] - init_usdt
            # profit = usdt_out - ask['price'] * amt
            if op_spot < ask['price']:
                amt_high = amt
            elif op_spot > ask['price']:
                amt_low = amt
            else:
                break
                
        op_state.swap(agent, tkn_buy='USDT', tkn_sell='DOT', buy_quantity=amt_low)
        op_spot = OmnipoolState.price(op_state, 'DOT', 'USDT')
        swaps.append({'price': ask['price'], 'amount': amt_low})
        usdt_out = agent.holdings['USDT'] - init_usdt
        profit += usdt_out - ask['price'] * amt_low
print(profit)
print(op_spot)
print(swaps)
    
    # def swap(
    #         self,
    #         agent: Agent,
    #         tkn_buy: str, tkn_sell: str,
    #         buy_quantity: float = 0,
    #         sell_quantity: float = 0,
    #         modify_imbalance: bool = True,  # this is a hack to avoid modifying the imbalance for arbitrager LRNA swaps,
    #         # since those would not actually be executed as LRNA swaps
    #         # note that we still apply the imbalance modification due to LRNA fee
    #         # collection, we just don't apply the imbalance modification from
    #         # the sale of LRNA back to the pool.
    # ):

sell
sell
77393.5067763552
3.74458182070966
[{'price': 3.7, 'amount': 100}, {'price': 3.74, 'amount': 167.2011879572528}]
